In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoConfig, TrainingArguments, Trainer
from datasets import load_dataset

checkpoint_md = 'google-bert/bert-base-uncased'
checkpoint_db = 'dair-ai/emotion'
model = AutoModel.from_pretrained(checkpoint_md)
data = load_dataset(checkpoint_db)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_md)
def tokenize(input):
    return tokenizer(input['text'], truncation=True, padding=True)
batch_data = data.map(tokenize, batched=True)

c:\Users\sgogo\python_code\venv_hugging_face\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model_ckpt = "sentence-transformers/all-MiniLM-L6-v2"
tokenize = AutoTokenizer.from_pretrained(model_ckpt)
embedding_model = SentenceTransformer(model_ckpt)

texts = [chunk.page_content for chunk in documents]
embeddings = embedding_model.encode(
    texts,
    convert_to_tensor=True,  # 若你之後要和 PyTorch 結合推薦設 True
    show_progress_bar=True
)
embeddings_np = embeddings.cpu().detach().numpy()

# 建立 FAISS 索引器
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)

# 建立一個對照表（index 到原始 document metadata）
id_to_doc = {i: documents[i] for i in range(len(documents))}



# query = "微創"
# query_embedding = embedding_model.encode(query, convert_to_tensor=False)
# D, I = index.search(np.array([query_embedding]), k=3)  # 找最近的3筆

# print(id_to_doc[4687].page_content)

# # 取出對應文件片段
# results = [id_to_doc[i] for i in I[0]]
# for doc in results:
#     print(doc.metadata.get("title", ""), doc.page_content[:100])

Batches: 100%|██████████| 556/556 [03:37<00:00,  2.56it/s]


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import util.util as util
import numpy as np
from transformers import pipeline
import chinese_converter

model_ckpt = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(model_ckpt)
chunk_path = "./chunks/chunks.json"
documents = util.load_document(chunk_path)
vector_path = "./vector_store/my_index.faiss"
index = faiss.read_index(vector_path)
llm = pipeline("text-generation", model="unsloth/Qwen2.5-3B")

query = "近期在屏東是否有發生磷酸液體洩漏事件"
query_embedding = embedding_model.encode(query, convert_to_tensor=False)
distances, indices = index.search(np.array([query_embedding]), k=3)

context = ""
for i in range(len(indices[0])):
    context = context + f"{i+1}. {documents[indices[0, i]].page_content[2:]}\n"

# 查詢
prompt = f"根據以下新聞片段回答問題：\n\n{context}\n問題：{query}\n回答："
prompt = chinese_converter.to_simplified(prompt)
print(f"question: {prompt}\n")

response = llm(prompt, max_length=300)
print(chinese_converter.to_traditional(response[0]["generated_text"]))

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  8.28it/s]
Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


question: 根据以下新闻片段回答问题：

1. 关服务资讯可至屏东县政府卫生局官网查询。
2. 市场人士指出，平实营区为台南东区的蛋黄精华区，区域生活圈拥有南纺购物中心、国宾影城、高雄荣总台南分院等，生活机能完善、交通便利。
3. 染科医师呼吁，面对即将到来的中秋节及国庆连假，南来北往免疫力容易下降，大人小孩都需保持良好的卫生习惯，有症状就须要就医，不要小病拖成大病

问题：近期在屏东是否有发生磷酸液体泄漏事件
回答：

根據以下新聞片段回答問題：

1. 關服務資訊可至屏東縣政府衛生局官網查詢。
2. 市場人士指出，平實營區為臺南東區的蛋黃精華區，區域生活圈擁有南紡購物中心、國賓影城、高雄榮總臺南分院等，生活機能完善、交通便利。
3. 染科醫師呼籲，面對即將到來的中秋節及國慶連假，南來北往免疫力容易下降，大人小孩都需保持良好的衛生習慣，有症狀就須要就醫，不要小病拖成大病

問題：近期在屏東是否有發生磷酸液體洩漏事件
回答：無

根據提供的新聞片段，近期在屏東沒有發生磷酸液體洩漏事件。新聞片段中提到的是屏東縣政府衛生局的官方網站上有關服務資訊的獲取方式，以及對於臺南東區市場的人士的建議，主要關注的是平實營區的生活圈、交通便利以及


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import util.util as util
import numpy as np

model_ckpt = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(model_ckpt)
chunk_path = "./chunks/chunks.json"
documents = util.load_document(chunk_path)
vector_path = "./vector_store/my_index.faiss"

# 讀回 index
index = faiss.read_index(vector_path)

# 查詢
query = "威爾姆氏腫瘤"
query_embedding = embedding_model.encode(query, convert_to_tensor=False)
distances, indices = index.search(np.array([query_embedding]), k=3)

for i in indices[0]:
    print(f"content({i}): {documents[i].page_content}")


content(5711): 。
・圖表看時事／讓8歲童擺脫洗腎人生 圖解亞洲首例分肝種腎、3難點一次看
8歲的吳小妹罹罕見癌症「威爾姆氏腫瘤」，兩顆腎臟嚴重變形，只得手術切除，多年以來靠洗腎續命
content(6580): 。
林邑璁指出，克雷伯氏肺炎桿菌引起的化膿性肝膿瘍是國內很常見的疾病，不同於西方國家的肝膿瘍多由於膽道病變引起，東亞國家盛行具有高毒性的克雷伯氏肺炎桿菌造成的肝膿瘍，疾病發生機轉認為是在腸道移生的克雷伯氏肺炎桿菌移行至肝臟造成化膿性病變，造成許多沒有肝膽病變的人得到肝膿瘍
content(13482): 。
但散發型庫賈氏症的進程相當迅速，患者通常於發病後3～12個月內死亡（統計值：中位數為 4 個月、平均值為 7 個月）。
台灣過去雖曾因國外牛肉進口事件引起爭議，但至今國內尚無發現有人罹患狂牛症（新型庫賈氏症）


In [1]:
from transformers import pipeline
model_ckpt = "openai/clip-vit-base-patch32"
llm = pipeline("zero-shot-image-classification", model=model_ckpt)

c:\Users\sgogo\python_code\venv_hugging_face_3-12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sgogo\python_code\venv_hugging_face_3-12\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sgogo\.cache\huggingface\hub\models--openai--clip-vit-base-patch32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admin